In [1]:
import pandas as pd
import openpyxl

In [2]:
df = pd.read_excel('GENeSYS-MOD_User_Input_Settings_v01_phe_05-09-2023.xlsx', sheet_name='Region_selection')

# Now you can work with the DataFrame (e.g., display, manipulate, analyze data)
print(df)

  Continent Country Country_Sublevel_1 Country_Sublevel_2  Region Selected
0    Europe      DE              DE-BY           DE-BY-UF                1
1    Europe      NO                NO1               NO11                1
2    Europe      NO                NO1                NO1                1
3    Europe      NO                 NO                 NO                0
4    Europe      DE              DE-BY              DE-BY                1
5    Europe      DE                 DE                 DE                1
6      Asia      CN                 CN                 CN                0
7      Asia      CN             CN-Cen             CN-Cen                1
8      Asia      CN             CN-Cen       CN-Cen-Henan                1


In [3]:
filtered_df = df[df.iloc[:, -1] == 1]

# Now 'filtered_df' contains only the entries with a 1 in the last field
print(filtered_df)

  Continent Country Country_Sublevel_1 Country_Sublevel_2  Region Selected
0    Europe      DE              DE-BY           DE-BY-UF                1
1    Europe      NO                NO1               NO11                1
2    Europe      NO                NO1                NO1                1
4    Europe      DE              DE-BY              DE-BY                1
5    Europe      DE                 DE                 DE                1
7      Asia      CN             CN-Cen             CN-Cen                1
8      Asia      CN             CN-Cen       CN-Cen-Henan                1


In [4]:
# Read the CSV file into a Pandas DataFrame, starting from row 5 and using ';' as the delimiter
second_df = pd.read_csv('csvs\Par_RegionalBaseYearProduction.csv', skiprows=4, delimiter=';')

# Now you have the data from the CSV file stored in the 'second_df' DataFrame
print(second_df)

     Region           Technology                  Fuel      2018
0        AT          HLR_Biomass  Heat_Low_Residential    71,694
1        AT         HLR_Hardcoal  Heat_Low_Residential     2,794
2        AT       HLR_Oil_Boiler  Heat_Low_Residential    57,279
3        AT          HLI_Biomass   Heat_Low_Industrial     2,659
4        AT         HLI_Hardcoal   Heat_Low_Industrial     0,275
...     ...                  ...                   ...       ...
1026    NO1  HLR_Direct_Electric  Heat_Low_Residential  61,14528
1027    NO2  HLR_Direct_Electric  Heat_Low_Residential  34,94016
1028    NO3  HLR_Direct_Electric  Heat_Low_Residential  20,38176
1029    NO4  HLR_Direct_Electric  Heat_Low_Residential  13,10256
1030    NO5  HLR_Direct_Electric  Heat_Low_Residential  16,01424

[1031 rows x 4 columns]


In [5]:
# Extract the unique values from the fourth entry (dimension) of the first DataFrame
unique_values_from_first_df = filtered_df.iloc[:, 3].unique()

# Filter the second DataFrame to keep only the rows where the first dimension is in the unique values from the first DataFrame
filtered_second_df = second_df[second_df.iloc[:, 0].isin(unique_values_from_first_df)]

# Now 'filtered_second_df' contains only the values where the first dimension can be found in the fourth entry of the first DataFrame
print(filtered_second_df)

     Region           Technology                  Fuel      2018
162      DE          HLR_Biomass  Heat_Low_Residential   251,332
163      DE         HLR_Hardcoal  Heat_Low_Residential    45,728
164      DE       HLR_Oil_Boiler  Heat_Low_Residential   681,617
165      DE          HLI_Biomass   Heat_Low_Industrial      5,19
166      DE         HLI_Hardcoal   Heat_Low_Industrial     5,076
...     ...                  ...                   ...       ...
1006    NO1           P_Gas_OCGT                 Power   0,16975
1011    NO1    CHP_Biomass_Solid                 Power    0,0428
1016    NO1  HLR_Heatpump_Aerial  Heat_Low_Residential   6,45414
1021    NO1  HLR_Heatpump_Ground  Heat_Low_Residential   0,33978
1026    NO1  HLR_Direct_Electric  Heat_Low_Residential  61,14528

[61 rows x 4 columns]


In [6]:
# Read the CSV file into a Pandas DataFrame, starting from row 5 and using ';' as the delimiter
third_df = pd.read_csv('csvs\Par_CapitalCost.csv', skiprows=4, delimiter=';')

# Now you have the data from the CSV file stored in the 'second_df' DataFrame
print(third_df)

   Region                Technology               2018               2020  \
0      DE         CHP_Biomass_Solid             2200,0             2090,0   
1      DE         CHP_Coal_Hardcoal             2750,0             2750,0   
2      DE          CHP_Coal_Lignite             2200,0             2200,0   
3      DE      CHP_Gas_CCGT_Natural  935,0000000000001  858,0000000000001   
4      DE       CHP_Gas_CCGT_Biogas  935,0000000000001  858,0000000000001   
5      DE       CHP_Gas_CCGT_SynGas  935,0000000000001  858,0000000000001   
6      DE     CHP_Biomass_Solid_CCS           6342,105           6342,105   
7      DE     CHP_Coal_Hardcoal_CCS             4125,0             4125,0   
8      DE      CHP_Coal_Lignite_CCS             3300,0             3300,0   
9      DE  CHP_Gas_CCGT_Natural_CCS             1402,5             1287,0   
10     DE   CHP_Gas_CCGT_Biogas_CCS             1402,5             1287,0   
11     DE     CHP_Hydrogen_FuelCell             1100,0            1081,25   

In [7]:
# Filter the second DataFrame to keep only the rows where the first dimension is in the unique values from the first DataFrame
filtered_third_df = third_df[third_df.iloc[:, 0].isin(unique_values_from_first_df)]

# Now 'filtered_second_df' contains only the values where the first dimension can be found in the fourth entry of the first DataFrame
print(filtered_third_df)

   Region                Technology               2018               2020  \
0      DE         CHP_Biomass_Solid             2200,0             2090,0   
1      DE         CHP_Coal_Hardcoal             2750,0             2750,0   
2      DE          CHP_Coal_Lignite             2200,0             2200,0   
3      DE      CHP_Gas_CCGT_Natural  935,0000000000001  858,0000000000001   
4      DE       CHP_Gas_CCGT_Biogas  935,0000000000001  858,0000000000001   
5      DE       CHP_Gas_CCGT_SynGas  935,0000000000001  858,0000000000001   
6      DE     CHP_Biomass_Solid_CCS           6342,105           6342,105   
7      DE     CHP_Coal_Hardcoal_CCS             4125,0             4125,0   
8      DE      CHP_Coal_Lignite_CCS             3300,0             3300,0   
9      DE  CHP_Gas_CCGT_Natural_CCS             1402,5             1287,0   
10     DE   CHP_Gas_CCGT_Biogas_CCS             1402,5             1287,0   
11     DE     CHP_Hydrogen_FuelCell             1100,0            1081,25   

In [8]:
# Create a Pandas Excel writer
with pd.ExcelWriter('output.xlsx', engine='openpyxl') as writer:
    # Write the first DataFrame to the Excel file
    df.to_excel(writer, sheet_name='FirstDataFrame', index=False)
    
    # Write the second filtered DataFrame to the Excel file
    filtered_second_df.to_excel(writer, sheet_name='FilteredSecondDataFrame', index=False)
    
    # Write the third filtered DataFrame to the Excel file
    filtered_third_df.to_excel(writer, sheet_name='FilteredThirdDataFrame', index=False)
